In [ ]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.types import DateType, TimestampType
from pyspark.sql.functions import date_format
import os
import concurrent.futures



In [ ]:
def get_df(
    spark_session,
    jdbc_url: str,
    user: str,
    password: str,
    sql_statement: str,
    driver: str = "com.ibm.db2.jcc.DB2Driver",
):
    spark_sql_statement = f"({sql_statement}) as tab"

    df_config = (
        spark_session.read.format("jdbc")
        .option("url", jdbc_url)
        .option("driver", driver)
        .option("user", user)
        .option("password", password)
        .option("dbtable", spark_sql_statement)
    )
    df = df_config.load()
    return df


In [ ]:
conf = SparkConf()
conf.set(
    "spark.jars",
    "/container-dir/JARs/db2jcc4.jar,/container-dir/JARs/db2jcc_license_cisuz.jar,/container-dir/JARs/db2jcc_license_cu.jar,",
)
conf.set("spark.driver.memory", "6g")  # 8g
spark = SparkSession.builder.config(conf=conf).getOrCreate()


---
# Get Some Data

In [ ]:
sql_statement = """
SELECT *
FROM BLAH.BLAH
"""
df = get_df(
    spark_session=spark,
    jdbc_url=r"jdbc:db2://the-db2:50001/BLUDB:sslConnection=true;sslTrustStoreLocation=/container-dir/truststore/ibm-truststore.jks;sslTrustStorePassword=changeit;",
    user=os.getenv("db_user"),
    password=os.getenv("db_pw"),
    sql_statement=sql_statement,
)
df_local = spark.createDataFrame(data=df_LOAD.collect(), schema=df_LOAD.schema)
df_local.createOrReplaceTempView("data_view")
print(df_local.head())
print(df_local.count())


In [ ]:
df_local.toPandas().to_csv("./results/data.csv")